### experiment protocol
#### run:
     source protocol.md
     
#### problem definition
In computer vision, error behavior can have significant effects on decision making. For instance, the wrong estimation of distance can cause a car to stop or in the worst case not stop in the event of an obstacle. Furthermore, false positives can cause failure in control and navigation algorithms that rely on a particular distribution. In our case, we proposed a method for containing the error in a way that will reduce the overall error and avoid momentary large errors.   
     
#### testcase description
1. start at a starting height where to potential features are visible
2. extract features
3. climb to a relative position
4. run findEssentialMat to get relative position without scale
5. run recoverPose based on the essential mat to extract rotation vector and translation
6. clean untracked features
7. update translation to the correct scale based on the high difference from stage 3
8. run triangulatePoints based on the two camera positions (stage 1 & 3)
9. move camera at rectangular trajectory looking downwards facing the features
10. run solvePnP to get camera position and orientation

#### type of solvePnP tested
1. solvePnP used in opencv
2. modified solvePnP based on least_square from scipy using the bounded trf method. representing the rotation with axis angle representation
3. modified solvePnP based on least_square from scipy using the bounded trf method. representing the rotation with Euler angles representation

#### methods
###### software 
We developed a software tailored to this specefic problem

##### simulation
points selected on agrid and created syntetic camera by projecting the points to the camera position. noise was added to the simulation proccess after the trangulation. This was done in order to create meeningful error in terms of accepted acuricy in trangulation along the Z axis (the hight of the camera)

##### RealWorld
###### Haedware
- Sony eye cam II
- sensor gy86 todo: ... ad info
- optitrack system for ground truth information

###### Software
We have developed asoftware in python to evaluate camera position based on the magnetometr readings and the accelerometer readings.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pickle
import numpy as np 


In [ ]:
def read_experiment_file(fname):
    ret=[]
    with open(fname,'rb') as fd:
        try:
            while 1:
                ret.append(pickle.load(fd))
        except EOFError:
            pass
    return ret

def plot_camdiff_pos(ax1,ax2,ex_data,ex_type='sim'):
    extract = lambda x: [(d[1][0],*d[1][2]) for d in ex_data if d[0]==x]
    cam_xyz=np.array(extract('camera'))
    if ex_type=='sim':
        cam_gt_xyz=np.array(extract('camera_gt'))
    elif ex_type=='liveopti':
        cam_gt_xyz= np.array([(d[1][0],*d[1][1].position) for d in ex_data if d[0]=='odata'])
        cam_gt_xyz = cam_gt_xyz[:,[0,3,1,2]]
        cam_gt_xyz[:,2]=-cam_gt_xyz[:,2]

    ax1.plot(cam_xyz[:,0]-cam_xyz[0,0],cam_xyz[:,1],'r')
    ax1.plot(cam_xyz[:,0]-cam_xyz[0,0],cam_xyz[:,2],'g')
    ax1.plot(cam_xyz[:,0]-cam_xyz[0,0],cam_xyz[:,3],'b')
    ax1.plot(cam_gt_xyz[:,0]-cam_xyz[0,0],cam_gt_xyz[:,1],'r',alpha=0.5)
    ax1.plot(cam_gt_xyz[:,0]-cam_xyz[0,0],cam_gt_xyz[:,2],'g',alpha=0.5)
    ax1.plot(cam_gt_xyz[:,0]-cam_xyz[0,0],cam_gt_xyz[:,3],'b',alpha=0.5)
    ax2.plot(cam_xyz[:,1],cam_xyz[:,2],'r')
    ax2.plot(cam_gt_xyz[:,1],cam_gt_xyz[:,2],'m')

def plot_experiment(data_file ,ex_type='sim'):
    data=read_experiment_file(data_file)
    ax1=plt.subplot(1,2,1)
    ax2=plt.subplot(1,2,2)
    plot_camdiff_pos(ax1,ax2,data,ex_type)
    ax2.axis('equal')

def plot_experiments(data_file1, data_file2 ,ex_type='sim'):
    data=read_experiment_file(data_file1)
    ax1=plt.subplot(2,2,1)
    ax2=plt.subplot(2,2,2)
    plot_camdiff_pos(ax1,ax2,data,ex_type)
    ax2.axis('equal')
    data=read_experiment_file(data_file2)
    ax1=plt.subplot(2,2,3)
    ax2=plt.subplot(2,2,4)
    plot_camdiff_pos(ax1,ax2,data,ex_type)
    ax2.axis('equal')

def read_3dpoints(fname):
    with open(fname,'rb') as fd:
        try:
            while 1:
                ret=pickle.load(fd)
                if ret[0]=='pts3d':
                    return ret[1][1]
        except EOFError:
            pass
    return None
    
def plot_3dpoints(fname,ax1,ax2,color='b'):
    pts=read_3dpoints(fname)
    ax1.plot(pts[:,0],pts[:,1],'+'+color)
    ax2.hist(pts[:,2],bins=20,range=(-.2,.2),alpha=0.5,color=color)
   

In [ ]:
plt.figure()
plot_experiments('out/sim_pnp_1_nonoise.pkl','out/sim_pnp_1_noise.pkl')
plt.figure()
ax1=plt.subplot(1,2,1)
ax2=plt.subplot(1,2,2)
plot_3dpoints('out/sim_pnp_1_nonoise.pkl',ax1,ax2,'b')
plot_3dpoints('out/sim_pnp_1_noise.pkl',ax1,ax2,'r')


In [ ]:
plt.figure()
plot_experiment('out/sim_pnp_2_noise_constrains.pkl')


In [ ]:
plt.figure()
plot_experiment('out/sim_pnp_3_noise_constrains.pkl')


In [ ]:
#plt.figure()
data_out=read_experiment_file('out/optilive12_pnp_1.pkl')
data_out[4][1][1].position


In [ ]:
plt.figure()
plot_experiments('out/optilive12_pnp_1.pkl','out/optilive12_pnp_3_constrains.pkl','liveopti')
plt.figure()
ax1=plt.subplot(1,2,1)
ax2=plt.subplot(1,2,2)
plot_3dpoints('out/optilive12_pnp_1.pkl',ax1,ax2,'b')


In [ ]:
plt.figure()
plot_experiments('out/optilive12_pnp_1.pkl','out/optilive12_pnp_2_constrains.pkl','liveopti')


In [ ]:
plt.figure()
plot_experiments('out/optilive12_pnp_3_constrains.pkl','out/optilive12_pnp_2_constrains.pkl','liveopti')
